<table width="100%" style="border:none">
  <tr>
    <td valign="top">
      <h1>GTEx Preprocessing</h1>
      <ul>
<li>Author: Jeremy Yang
<li>Required: Python3, Pandas
<li>Clean, tidy, reshape RNAseq expression data.
<li>Save aggregated-samples median TPM file for downstream co-expression analysis.
<li>Save expression profiles (exfiles) TPM file for downstream co-expression analysis.
      </ul>
    </td>
    <td align="right">
        <p>UNM Translational Informatics Division</p>
<img style="float:right" width="100" src="https://brand.unm.edu/logos/unm-logo-mark.jpg" alt="UNM Logo" />
    </td>
  </tr>
  </table>
 

In [62]:
import sys,os,re,time,io
import urllib.request
import google.colab
import numpy,scipy
import pandas as pd
print(f"Python: {sys.version.split()[0]}; Pandas: {pandas.__version__}; Scipy: {scipy.__version__} ; Numpy: {numpy.__version__}")

Python: 3.6.9; Pandas: 1.1.5; Scipy: 1.4.1 ; Numpy: 1.19.5


### Download subjects datafile:
GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.txt

In [63]:
url = "https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SubjectPhenotypesDS.txt"
subjects = pd.read_csv(url, "\t")
print(f"dataset nrows: {subjects.shape[0]} ; ncols: {subjects.shape[1]}:")

dataset nrows: 980 ; ncols: 4:


In [64]:
subjects.head()

,SUBJID,SEX,AGE,DTHHRDY
0,GTEX-1117F,2,60-69,4.0
1,GTEX-111CU,1,50-59,0.0
2,GTEX-111FC,1,60-69,1.0
3,GTEX-111VG,1,60-69,3.0
4,GTEX-111YS,1,60-69,0.0


In [65]:
subjects.AGE.value_counts().sort_index()

20-29     84
30-39     78
40-49    153
50-59    315
60-69    317
70-79     33
Name: AGE, dtype: int64

### Download samples datafile:
new: GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt

In [66]:
url = "https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt"
samples = pd.read_csv(url, sep='\t')
samples = samples[['SAMPID', 'SMATSSCR', 'SMTS', 'SMTSD', 'SMUBRID']]
print("dataset nrows: %d ; ncols: %d:"%(samples.shape[0],samples.shape[1]))

dataset nrows: 22951 ; ncols: 5:


 * SMTS = Tissue Type (parent of SMTSD)
 * SMTSD = Tissue Type, more specific
 * SMATSSCR = Autolysis Score, 0=None, 1=Mild, 2=Moderate, 3=Severe
 * Note that other sample attributes may be suitable for quality criteria.
 * SMUBRID = Uberon ID, anatomical location
 * SUBJID is first two hyphen-delimted fields of SAMPID.

In [67]:
samples['SUBJID'] = samples.SAMPID.str.extract('^([^-]+-[^-]+)-', expand=True)
smtsd_orig = samples.SMTSD.unique()
samples.head()

,SAMPID,SMATSSCR,SMTS,SMTSD,SMUBRID,SUBJID
0,GTEX-1117F-0003-SM-58Q7G,NaN,Blood,Whole Blood,0013756,GTEX-1117F
1,GTEX-1117F-0003-SM-5DWSB,NaN,Blood,Whole Blood,0013756,GTEX-1117F
2,GTEX-1117F-0003-SM-6WBT7,NaN,Blood,Whole Blood,0013756,GTEX-1117F
3,GTEX-1117F-0011-R10a-SM-AHZ7F,NaN,Brain,Brain - Frontal Cortex (BA9),0009834,GTEX-1117F
4,GTEX-1117F-0011-R10b-SM-CYKQ8,NaN,Brain,Brain - Frontal Cortex (BA9),0009834,GTEX-1117F


In [68]:
print("Tissue types: %s"%(str(set(smtsd_orig))))

Tissue types: {'Esophagus - Muscularis', 'Esophagus - Mucosa', 'Brain - Amygdala', 'Colon - Sigmoid', 'Ovary', 'Skin - Sun Exposed (Lower leg)', 'Esophagus - Gastroesophageal Junction', 'Kidney - Medulla', 'Artery - Tibial', 'Brain - Cerebellum', 'Fallopian Tube', 'Heart - Left Ventricle', 'Small Intestine - Terminal Ileum', 'Cervix - Endocervix', 'Artery - Coronary', 'Spleen', 'Adipose - Subcutaneous', 'Cells - Cultured fibroblasts', 'Nerve - Tibial', 'Lung', 'Colon - Transverse', 'Vagina', 'Brain - Nucleus accumbens (basal ganglia)', 'Brain - Hippocampus', 'Brain - Cerebellar Hemisphere', 'Brain - Anterior cingulate cortex (BA24)', 'Brain - Substantia nigra', 'Liver', 'Whole Blood', 'Bladder', 'Brain - Caudate (basal ganglia)', 'Cells - Leukemia cell line (CML)', 'Adrenal Gland', 'Brain - Putamen (basal ganglia)', 'Adipose - Visceral (Omentum)', 'Muscle - Skeletal', 'Brain - Cortex', 'Cells - EBV-transformed lymphocytes', 'Heart - Atrial Appendage', 'Breast - Mammary Tissue', 'Pancre

In [69]:
print("Counts: SAMPID: %d; SMTS: %d; SMTSD: %d; SUBJID: %d"%(
      samples.SAMPID.nunique(), samples.SMTS.nunique(), samples.SMTSD.nunique(), samples.SUBJID.nunique()))

Counts: SAMPID: 22951; SMTS: 31; SMTSD: 55; SUBJID: 980


### Remove samples with high degree of autolysis (self-digestion).
The destruction of organism cells or tissues by the organisms’ own enzymes or processes.
0=None, 1=Mild, 2=Moderate, 3=Severe

In [70]:
samples.SMATSSCR.value_counts(dropna=False).sort_index()

0.0     3554
1.0    10410
2.0     1582
3.0      193
NaN     7212
Name: SMATSSCR, dtype: int64

In [71]:
samples = samples[(samples.SMATSSCR != 3) & (samples.SMATSSCR != 2)]
print("Counts: SAMPID: %d; SMTS: %d; SMTSD: %d; SUBJID: %d"%(
      samples.SAMPID.nunique(), samples.SMTS.nunique(), samples.SMTSD.nunique(), samples.SUBJID.nunique()))

Counts: SAMPID: 21176; SMTS: 31; SMTSD: 55; SUBJID: 980


### Clean & tidy cols. 

In [72]:
samples.loc[(samples.SMTS.str.strip() == '') & samples.SMTSD.str.startswith("Skin -"), 'SMTS'] = 'Skin'

In [73]:
(samples.SMTS+" : "+samples.SMTSD).value_counts().sort_index()

Adipose Tissue : Adipose - Subcutaneous                752
Adipose Tissue : Adipose - Visceral (Omentum)          560
Adrenal Gland : Adrenal Gland                          209
Bladder : Bladder                                        8
Blood : Cells - EBV-transformed lymphocytes            192
Blood : Whole Blood                                   3288
Blood Vessel : Artery - Aorta                          444
Blood Vessel : Artery - Coronary                       251
Blood Vessel : Artery - Tibial                         757
Bone Marrow : Cells - Leukemia cell line (CML)         217
Brain : Brain - Amygdala                               177
Brain : Brain - Anterior cingulate cortex (BA24)       213
Brain : Brain - Caudate (basal ganglia)                291
Brain : Brain - Cerebellar Hemisphere                  263
Brain : Brain - Cerebellum                             226
Brain : Brain - Cortex                                 268
Brain : Brain - Frontal Cortex (BA9)                   4

### MERGE samples with subjects:

In [74]:
samples = pd.merge(samples, subjects, how='inner', on='SUBJID')
samples.head()

,SAMPID,SMATSSCR,SMTS,SMTSD,SMUBRID,SUBJID,SEX,AGE,DTHHRDY
0,GTEX-1117F-0003-SM-58Q7G,NaN,Blood,Whole Blood,0013756,GTEX-1117F,2,60-69,4.0
1,GTEX-1117F-0003-SM-5DWSB,NaN,Blood,Whole Blood,0013756,GTEX-1117F,2,60-69,4.0
2,GTEX-1117F-0003-SM-6WBT7,NaN,Blood,Whole Blood,0013756,GTEX-1117F,2,60-69,4.0
3,GTEX-1117F-0011-R10a-SM-AHZ7F,NaN,Brain,Brain - Frontal Cortex (BA9),0009834,GTEX-1117F,2,60-69,4.0
4,GTEX-1117F-0011-R10b-SM-CYKQ8,NaN,Brain,Brain - Frontal Cortex (BA9),0009834,GTEX-1117F,2,60-69,4.0


In [75]:
print(f"Counts: SAMPID: {samples.SAMPID.nunique()}; SMTS: {samples.SMTS.nunique()}; SMTSD: {samples.SMTSD.nunique()}; SUBJID: {samples.SUBJID.nunique()}")

Counts: SAMPID: 21176; SMTS: 31; SMTSD: 55; SUBJID: 980


### Keep only samples from healthier subjects (and remove NAs): 
(DTHHRDY = Hardy Scale)
Death classification based on the 4-point Hardy Scale:
1) Violent and fast death Deaths due to accident, blunt force trauma or suicide, terminal phase estimated at < 10 min. 
2) Fast death of natural causes Sudden unexpected deaths of people who had been reasonably healthy, after a terminal phase estimated at < 1 hr (with sudden death from a myocardial infarction as a model cause of death for this category) 
3) Intermediate death Death after a terminal phase of 1 to 24 hrs (not classifiable as 2 or 4); patients who were ill but death was unexpected 
4) Slow death Death after a long illness, with a terminal phase longer than 1 day (commonly cancer or chronic pulmonary disease); deaths that are not unexpected 
0) Ventilator Case All cases on a ventilator immediately before death.


In [76]:
samples.DTHHRDY.value_counts(sort=True, dropna=False).sort_index()

0.0    11131
1.0      832
2.0     5331
3.0     1030
4.0     2435
NaN      417
Name: DTHHRDY, dtype: int64

In [77]:
samples = samples[samples.DTHHRDY<=2]
samples.DTHHRDY.value_counts(sort=True, dropna=False).sort_index()

0.0    11131
1.0      832
2.0     5331
Name: DTHHRDY, dtype: int64

In [78]:
smtsd_final = samples.SMTSD.unique()
smtsd_lost = set(smtsd_orig) - set(smtsd_final)
print("Tissue types lost: "+str(smtsd_lost))
print(f"Counts: SAMPID: {samples.SAMPID.nunique()}; SMTS: {samples.SMTS.nunique()}; SMTSD: {samples.SMTSD.nunique()}; SUBJID: {samples.SUBJID.nunique()}")

Tissue types lost: {'Cells - Leukemia cell line (CML)'}
Counts: SAMPID: 17294; SMTS: 30; SMTSD: 54; SUBJID: 785


### Clean & tidy:

In [79]:
samples.SEX = samples.SEX.apply(lambda x: 'F' if x==2 else 'M' if x==1 else None)
print(samples.SEX.value_counts(sort=True, dropna=False).sort_index())
if (samples.SEX.isna().sum()>0):
  samples.dropna(subset=['SEX'], inplace=True)
print(f"Counts: SAMPID: {samples.SAMPID.nunique()}; SMTS: {samples.SMTS.nunique()}; SMTSD: {samples.SMTSD.nunique()}; SUBJID: {samples.SUBJID.nunique()}")

F     5761
M    11533
Name: SEX, dtype: int64
Counts: SAMPID: 17294; SMTS: 30; SMTSD: 54; SUBJID: 785


### Download GENE TPMs (full or demo subset)
Full file is ~56k rows, 2.6GB uncompressed.  Demo ~5k rows.

In [106]:
url = "https://storage.googleapis.com/gtex_analysis_v8/rna_seq_data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct.gz"
t0 = time.time()
rnaseq = pd.read_table(url, compression="gzip", sep="\t", skiprows=2, nrows=1000)
print(f"dataset nrows: {rnaseq.shape[0]} ; ncols: {rnaseq.shape[1]}:")
print(f"Elapsed: {time.time()-t0:.2f}s")

dataset nrows: 10000 ; ncols: 17384:
Elapsed: 137.97s


In [107]:
rnaseq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 17384 entries, Name to GTEX-ZZPU-2726-SM-5NQ8O
dtypes: float64(17382), object(2)
memory usage: 1.3+ GB


In [108]:
rnaseq = rnaseq.drop(columns=['Description'])
rnaseq = rnaseq.rename(columns={'Name':'ENSG'})
rnaseq.columns

Index(['ENSG', 'GTEX-1117F-0226-SM-5GZZ7', 'GTEX-1117F-0426-SM-5EGHI',
       'GTEX-1117F-0526-SM-5EGHJ', 'GTEX-1117F-0626-SM-5N9CS',
       'GTEX-1117F-0726-SM-5GIEN', 'GTEX-1117F-1326-SM-5EGHH',
       'GTEX-1117F-2426-SM-5EGGH', 'GTEX-1117F-2526-SM-5GZY6',
       'GTEX-1117F-2826-SM-5GZXL',
       ...
       'GTEX-ZZPU-1126-SM-5N9CW', 'GTEX-ZZPU-1226-SM-5N9CK',
       'GTEX-ZZPU-1326-SM-5GZWS', 'GTEX-ZZPU-1426-SM-5GZZ6',
       'GTEX-ZZPU-1826-SM-5E43L', 'GTEX-ZZPU-2126-SM-5EGIU',
       'GTEX-ZZPU-2226-SM-5EGIV', 'GTEX-ZZPU-2426-SM-5E44I',
       'GTEX-ZZPU-2626-SM-5E45Y', 'GTEX-ZZPU-2726-SM-5NQ8O'],
      dtype='object', length=17383)

### MELT: One row per ENSG+SAMPID+TPM triplet:
Easier to handle but ~3x storage.

In [109]:
rnaseq = rnaseq.melt(id_vars = "ENSG", var_name = "SAMPID", value_name = "TPM")

In [110]:
rnaseq.head()

,ENSG,SAMPID,TPM
0,ENSG00000223972.5,GTEX-1117F-0226-SM-5GZZ7,0.00000
1,ENSG00000227232.5,GTEX-1117F-0226-SM-5GZZ7,8.76400
2,ENSG00000278267.1,GTEX-1117F-0226-SM-5GZZ7,0.00000
3,ENSG00000243485.5,GTEX-1117F-0226-SM-5GZZ7,0.07187
4,ENSG00000237613.2,GTEX-1117F-0226-SM-5GZZ7,0.00000


In [111]:
rnaseq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173820000 entries, 0 to 173819999
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   ENSG    object 
 1   SAMPID  object 
 2   TPM     float64
dtypes: float64(1), object(2)
memory usage: 3.9+ GB


### Read and merge gene symbols.
File from https://www.ensembl.org/biomart, dataset human genes, fields Gene stable ID, Gene stable ID version,  NCBI gene ID, HGNC symbol.

In [112]:
print('Upload Biomart ENSG2NCBI genes datafile: ')
uploaded = google.colab.files.upload()
fn = list(uploaded.keys())[0]
print('Uploaded "{name}" with {length} bytes'.format(name=fn, length=len(uploaded[fn])))
genes = pandas.read_csv(io.StringIO(uploaded[fn].decode('utf8')), sep='\t', usecols=[1,2,3], na_values=[''], dtype={2:str})
genes.columns = ['ENSG','NCBI','HGNC']
genes.dropna(inplace=True)

Upload Biomart ENSG2NCBI genes datafile: 


Saving biomart_ENSG2NCBI_human.tsv to biomart_ENSG2NCBI_human (2).tsv
Uploaded "biomart_ENSG2NCBI_human.tsv" with 2838104 bytes


In [113]:
genes.head()

,ENSG,NCBI,HGNC
5,ENSG00000198888.2,4535,MT-ND1
9,ENSG00000198763.3,4536,MT-ND2
15,ENSG00000198804.2,4512,MT-CO1
18,ENSG00000198712.1,4513,MT-CO2
20,ENSG00000228253.1,4509,MT-ATP8


In [114]:
rnaseq = pd.merge(rnaseq, genes, on='ENSG', how='inner')

In [115]:
rnaseq.head()

,ENSG,SAMPID,TPM,NCBI,HGNC
0,ENSG00000187642.9,GTEX-1117F-0226-SM-5GZZ7,1.2820,84808,PERM1
1,ENSG00000187642.9,GTEX-1117F-0426-SM-5EGHI,19.6900,84808,PERM1
2,ENSG00000187642.9,GTEX-1117F-0526-SM-5EGHJ,2.2660,84808,PERM1
3,ENSG00000187642.9,GTEX-1117F-0626-SM-5N9CS,0.7919,84808,PERM1
4,ENSG00000187642.9,GTEX-1117F-0726-SM-5GIEN,47.8000,84808,PERM1


### Remove genes in pseudoautosomal regions (PAR) of chromosome Y ("ENSGR").

In [116]:
n_ensgr = rnaseq.ENSG.str.startswith('ENSGR').sum()
print(f"ENSGR gene TPMs: {n_ensgr} ({100*n_ensgr/rnaseq.shape[0]:.2f}%)")

ENSGR gene TPMs: 0 (0.00%)


In [91]:
rnaseq = rnaseq[~rnaseq.ENSG.str.startswith('ENSGR')]

### Merge with samples:

In [117]:
rnaseq = pd.merge(rnaseq, samples, how="inner", on="SAMPID")
rnaseq = rnaseq.reset_index(drop=True)

In [118]:
rnaseq.head()

,ENSG,SAMPID,TPM,NCBI,HGNC,SMATSSCR,SMTS,SMTSD,SMUBRID,SUBJID,SEX,AGE,DTHHRDY
0,ENSG00000187642.9,GTEX-111CU-0126-SM-5GZWZ,0.6690,84808,PERM1,0.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-111CU,M,50-59,0.0
1,ENSG00000131591.17,GTEX-111CU-0126-SM-5GZWZ,4.0280,54991,C1orf159,0.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-111CU,M,50-59,0.0
2,ENSG00000184163.3,GTEX-111CU-0126-SM-5GZWZ,0.7839,388581,C1QTNF12,0.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-111CU,M,50-59,0.0
3,ENSG00000162576.16,GTEX-111CU-0126-SM-5GZWZ,27.3100,54587,MXRA8,0.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-111CU,M,50-59,0.0
4,ENSG00000175756.13,GTEX-111CU-0126-SM-5GZWZ,74.1500,54998,AURKAIP1,0.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-111CU,M,50-59,0.0


In [119]:
rnaseq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9501684 entries, 0 to 9501683
Data columns (total 13 columns):
 #   Column    Dtype  
---  ------    -----  
 0   ENSG      object 
 1   SAMPID    object 
 2   TPM       float64
 3   NCBI      object 
 4   HGNC      object 
 5   SMATSSCR  float64
 6   SMTS      object 
 7   SMTSD     object 
 8   SMUBRID   object 
 9   SUBJID    object 
 10  SEX       object 
 11  AGE       object 
 12  DTHHRDY   float64
dtypes: float64(3), object(10)
memory usage: 942.4+ MB


In [120]:
for i,smtsd in enumerate(rnaseq.SMTSD.sort_values().unique()):
  print(f"{i+1}. {smtsd}")

1. Adipose - Subcutaneous
2. Adipose - Visceral (Omentum)
3. Adrenal Gland
4. Artery - Aorta
5. Artery - Coronary
6. Artery - Tibial
7. Bladder
8. Brain - Amygdala
9. Brain - Anterior cingulate cortex (BA24)
10. Brain - Caudate (basal ganglia)
11. Brain - Cerebellar Hemisphere
12. Brain - Cerebellum
13. Brain - Cortex
14. Brain - Frontal Cortex (BA9)
15. Brain - Hippocampus
16. Brain - Hypothalamus
17. Brain - Nucleus accumbens (basal ganglia)
18. Brain - Putamen (basal ganglia)
19. Brain - Spinal cord (cervical c-1)
20. Brain - Substantia nigra
21. Breast - Mammary Tissue
22. Cells - Cultured fibroblasts
23. Cells - EBV-transformed lymphocytes
24. Cervix - Ectocervix
25. Cervix - Endocervix
26. Colon - Sigmoid
27. Colon - Transverse
28. Esophagus - Gastroesophageal Junction
29. Esophagus - Mucosa
30. Esophagus - Muscularis
31. Fallopian Tube
32. Heart - Atrial Appendage
33. Heart - Left Ventricle
34. Kidney - Cortex
35. Kidney - Medulla
36. Liver
37. Lung
38. Minor Salivary Gland
39. 

### Remove data for gene-tissue pairs with all zero expression.

In [121]:
maxtpm_0 = (rnaseq[['ENSG', 'SMTSD', 'TPM']].groupby(by=['ENSG','SMTSD'], as_index=True).max() == 0).rename(columns={'TPM':'maxtpm_0'})
maxtpm_0.maxtpm_0.value_counts()

False    38267
True       289
Name: maxtpm_0, dtype: int64

In [122]:
maxtpm_0.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 38556 entries, ('ENSG00000003137.8', 'Adipose - Subcutaneous') to ('ENSG00000284308.1', 'Whole Blood')
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   maxtpm_0  38556 non-null  bool 
dtypes: bool(1)
memory usage: 156.7+ KB


In [123]:
rnaseq = pd.merge(rnaseq, maxtpm_0, left_on=['ENSG', 'SMTSD'], right_index=True)
rnaseq.head()

,ENSG,SAMPID,TPM,NCBI,HGNC,SMATSSCR,SMTS,SMTSD,SMUBRID,SUBJID,SEX,AGE,DTHHRDY,maxtpm_0
0,ENSG00000187642.9,GTEX-111CU-0126-SM-5GZWZ,0.6690,84808,PERM1,0.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-111CU,M,50-59,0.0,False
22227,ENSG00000187642.9,GTEX-111YS-0126-SM-5987T,0.3483,84808,PERM1,1.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-111YS,M,60-69,0.0,False
42303,ENSG00000187642.9,GTEX-1122O-0326-SM-5H124,0.1945,84808,PERM1,0.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-1122O,F,60-69,0.0,False
81021,ENSG00000187642.9,GTEX-117YX-0126-SM-5EGH5,0.7508,84808,PERM1,1.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-117YX,M,50-59,0.0,False
115437,ENSG00000187642.9,GTEX-11DXX-0126-SM-5EGH7,0.2936,84808,PERM1,0.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-11DXX,F,60-69,0.0,False


In [124]:
rnaseq = rnaseq[~rnaseq['maxtpm_0']]
rnaseq.drop(columns=['maxtpm_0'], inplace=True)
rnaseq.head()

,ENSG,SAMPID,TPM,NCBI,HGNC,SMATSSCR,SMTS,SMTSD,SMUBRID,SUBJID,SEX,AGE,DTHHRDY
0,ENSG00000187642.9,GTEX-111CU-0126-SM-5GZWZ,0.6690,84808,PERM1,0.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-111CU,M,50-59,0.0
22227,ENSG00000187642.9,GTEX-111YS-0126-SM-5987T,0.3483,84808,PERM1,1.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-111YS,M,60-69,0.0
42303,ENSG00000187642.9,GTEX-1122O-0326-SM-5H124,0.1945,84808,PERM1,0.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-1122O,F,60-69,0.0
81021,ENSG00000187642.9,GTEX-117YX-0126-SM-5EGH5,0.7508,84808,PERM1,1.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-117YX,M,50-59,0.0
115437,ENSG00000187642.9,GTEX-11DXX-0126-SM-5EGH7,0.2936,84808,PERM1,0.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-11DXX,F,60-69,0.0


In [125]:
rnaseq.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9483320 entries, 0 to 7750769
Data columns (total 13 columns):
 #   Column    Dtype  
---  ------    -----  
 0   ENSG      object 
 1   SAMPID    object 
 2   TPM       float64
 3   NCBI      object 
 4   HGNC      object 
 5   SMATSSCR  float64
 6   SMTS      object 
 7   SMTSD     object 
 8   SMUBRID   object 
 9   SUBJID    object 
 10  SEX       object 
 11  AGE       object 
 12  DTHHRDY   float64
dtypes: float64(3), object(10)
memory usage: 1012.9+ MB


### Remove data for gene-tissue pairs not present in both sexes. (This removes most sex specific tissues.)

In [126]:
sex_count = (rnaseq[['ENSG', 'SMTSD', 'SEX']].groupby(by=['ENSG','SMTSD'], as_index=True).nunique()).rename(columns={'SEX':'sex_count'})
print(sex_count.sex_count.value_counts())

2    32003
1     6264
Name: sex_count, dtype: int64


In [127]:
sex_count_is_2 = (rnaseq[['ENSG', 'SMTSD', 'SEX']].groupby(by=['ENSG','SMTSD'], as_index=True).nunique()==2).rename(columns={'SEX':'ok'})
print(sex_count_is_2.ok.value_counts())

True     32003
False     6264
Name: ok, dtype: int64


In [128]:
sex_count.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 38267 entries, ('ENSG00000003137.8', 'Adipose - Subcutaneous') to ('ENSG00000284308.1', 'Whole Blood')
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   sex_count  38267 non-null  int64
dtypes: int64(1)
memory usage: 417.2+ KB


In [129]:
rnaseq = pd.merge(rnaseq, sex_count, left_on=['ENSG', 'SMTSD'], right_index=True, how="inner")
rnaseq.head()

,ENSG,SAMPID,TPM,NCBI,HGNC,SMATSSCR,SMTS,SMTSD,SMUBRID,SUBJID,SEX,AGE,DTHHRDY,sex_count
0,ENSG00000187642.9,GTEX-111CU-0126-SM-5GZWZ,0.6690,84808,PERM1,0.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-111CU,M,50-59,0.0,2
22227,ENSG00000187642.9,GTEX-111YS-0126-SM-5987T,0.3483,84808,PERM1,1.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-111YS,M,60-69,0.0,2
42303,ENSG00000187642.9,GTEX-1122O-0326-SM-5H124,0.1945,84808,PERM1,0.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-1122O,F,60-69,0.0,2
81021,ENSG00000187642.9,GTEX-117YX-0126-SM-5EGH5,0.7508,84808,PERM1,1.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-117YX,M,50-59,0.0,2
115437,ENSG00000187642.9,GTEX-11DXX-0126-SM-5EGH7,0.2936,84808,PERM1,0.0,Adrenal Gland,Adrenal Gland,0002369,GTEX-11DXX,F,60-69,0.0,2


In [ ]:
rnaseq = rnaseq[rnaseq['sex_count'] == 2]
rnaseq.drop(columns=['sex_count'], inplace=True)

In [ ]:
rnaseq.info()

In [ ]:
rnaseq.SMTSD.value_counts()

### Remove mammary tissue (partially sex-specific).

In [ ]:
smtsd_breast = "Breast - Mammary Tissue"
rnaseq = rnaseq[rnaseq.SMTSD!=smtsd_breast]


### Aggregate samples, compute median TPM by gene+tissue+sex+age:

In [130]:
rnaseq = rnaseq[['ENSG', 'SMTSD', 'SEX', 'AGE', 'TPM']].groupby(by=['ENSG', 'SMTSD', 'SEX', 'AGE'], as_index=False).median()
print(rnaseq.shape)
rnaseq.head()

(401596, 5)


,ENSG,SMTSD,SEX,AGE,TPM
0,ENSG00000003137.8,Adipose - Subcutaneous,F,20-29,40.285
1,ENSG00000003137.8,Adipose - Subcutaneous,F,30-39,33.960
2,ENSG00000003137.8,Adipose - Subcutaneous,F,40-49,36.910
3,ENSG00000003137.8,Adipose - Subcutaneous,F,50-59,41.530
4,ENSG00000003137.8,Adipose - Subcutaneous,F,60-69,36.340


### Aggregate samples, compute median TPM by gene+tissue+sex:

In [ ]:
rnaseq = rnaseq[['ENSG', 'SMTSD', 'SEX', 'TPM']].groupby(by=['ENSG','SMTSD','SEX'], as_index=False).median()
print(rnaseq.shape)
rnaseq.head()

### Save median TPMs file for analysis, 1-row per gene+tissue+sex:

In [ ]:
rnaseq.round(3).to_csv('gtex_rnaseq_prep_median.tsv', sep='\t', index=False)
google.colab.files.download('gtex_rnaseq_prep_median.tsv')

### Pivot TPMs to generate gene profiles:

In [ ]:
tissues = pd.Series(pd.unique(rnaseq.SMTSD.sort_values()))


In [ ]:
rnaseq_f = rnaseq[rnaseq.SEX=='F'].drop(columns=['SEX'])
rnaseq_f = rnaseq_f[['ENSG','SMTSD','TPM']]
exfiles_f = rnaseq_f.pivot(index='ENSG', columns='SMTSD')
exfiles_f.columns = exfiles_f.columns.get_level_values(1)
exfiles_f = exfiles_f.reset_index(drop=False)
exfiles_f['SEX'] = 'F'
exfiles_f.head()

In [ ]:
rnaseq_m = rnaseq[rnaseq.SEX=='M'].drop(columns=['SEX'])
rnaseq_m = rnaseq_m[['ENSG','SMTSD','TPM']]
exfiles_m = rnaseq_m.pivot(index='ENSG', columns='SMTSD')
exfiles_m.columns = exfiles_m.columns.get_level_values(1)
exfiles_m = exfiles_m.reset_index(drop=False)
exfiles_m['SEX'] = 'M'
exfiles_m.head()

In [ ]:
exfiles = pandas.concat([exfiles_f, exfiles_m])
cols = ['ENSG', 'SEX']+tissues.tolist()
exfiles = exfiles[cols]
exfiles.head()

In [ ]:
exfiles.info()

### Save expression profiles:

In [ ]:
exfiles.round(3).to_csv('exfiles_eps.tsv', sep='\t', index=False)
google.colab.files.download('exfiles_eps.tsv')